<a href="https://colab.research.google.com/github/frtrigg5/A-new-signature-model/blob/main/ModelConstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FBM Experiment

In [1]:
import sys
import os
# sys.path.append('/rds/general/user/ll1917/home/esig/gp-esig-classifier') # to add when running on remote Jupyter server
# os.chdir('/rds/general/user/ll1917/home/esig/gp-esig-classifier') # to add when running on remote Jupyter server

import torch
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

c:\Users\ll1917\Anaconda3\envs\esig-gp-esig-classifier-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
begin, end, number, division, dim = 0, 1, 100, 1, 1
'''
begin = first time steps
end = last known time steps
division = 1 means we are taking the middle points as new time instants -> L2 = L1 - 1
number = L1
dim = 1 - one dimensional
'''

# generating the time steps
known_times = torch.linspace(begin, end, number)
new_times = torch.zeros(division*(number-1))
for i in range(0,(number-1)):
  new_times[(division*i):(division*(i+1))] = torch.linspace(known_times[i], known_times[i+1], (division+2))[1:(1 + division)]

# Length of known values and new values
L1 = known_times.shape[0]
L2 = new_times.shape[0]

timesteps = torch.cat((known_times, new_times),axis=0)
timesteps_sorted, order = torch.sort(timesteps)

extended_order = torch.zeros(dim*order.size(0))
for i in range(order.size(0)):
  extended_order[(i*dim):((i+1)*dim)] = torch.arange(order[i]*dim, (order[i] + 1)*dim)

In [3]:
from lib.data.synthetic import BM_sample, FBM_sample

# construction of the dataset
trShape, vlShape, testShape = 1000, 400, 600
H = 0.26 # Hurst exponent

known_times = np.linspace(begin, end, number) # istanti temporali noti
div = 1 # quanti nuovi istanti temporali prendere tra due istanti noti
new_times = np.zeros(div*(number - 1))
for i in range((number-1)):
  new_times[(div*i):(div*(i+1))] = np.linspace(known_times[i], known_times[i+1], (div+2))[1:(1+div)]

L1 = known_times.size
L2 = new_times.size

timestamps = np.concatenate((known_times, new_times), axis=0)
# time series train
dataset_value = np.zeros(shape=[trShape, number])
seed = 0
for i in range(trShape//2):
  dataset_value[i] = BM_sample(begin, end, number, seed=seed+i)[0]
  dataset_value[i+trShape//2] = FBM_sample(begin, end, number, H, seed=seed+i)[0]

# time series validation
dataset_value2 = np.zeros(shape=[vlShape, number])
seed = trShape//2
for i in range(vlShape//2):
  dataset_value2[i] = BM_sample(begin, end, number, seed=seed+i)[0]
  dataset_value2[i+vlShape//2] = FBM_sample(begin, end, number, H, seed=seed+i)[0]

# time series test
dataset_value3 = np.zeros(shape=[testShape, number])
seed = trShape//2 + vlShape//2
for i in range(testShape//2):
  dataset_value3[i] = BM_sample(begin, end, number, seed=seed+i)[0]
  dataset_value3[i+testShape//2] = FBM_sample(begin, end, number, H, seed=seed+i)[0]
 
# adding known and unknown time stamps
time_data = np.zeros((trShape, L1 + L2))
for i in range(trShape):
  time_data[i] = timestamps

time_data2 = np.zeros((vlShape, L1 + L2))
for i in range(vlShape):
  time_data2[i] = timestamps

time_data3 = np.zeros((testShape, L1 + L2))
for i in range(testShape):
  time_data3[i] = timestamps  

# full dataset train
dataset = np.concatenate((time_data, dataset_value), axis=-1) # full dataset
dataset = dataset.astype('float32')

# full dataset validation
dataset2 = np.concatenate((time_data2, dataset_value2), axis=-1) # full dataset
dataset2 = dataset2.astype('float32')

# full dataset test
dataset3 = np.concatenate((time_data3, dataset_value3), axis=-1) # full dataset
dataset3 = dataset3.astype('float32')

# label construction
y = np.zeros(trShape, dtype='uint8') # label 0 for Brownian Motion, 1 for FBM
y[trShape//2:] = 1

#label di validation
y2 = np.zeros(vlShape, dtype='uint8')
y2[vlShape//2:] = 1

#label di test
y3 = np.zeros(testShape, dtype='uint8') 
y3[testShape//2:] = 1

In [4]:
from torch.utils.data import DataLoader, TensorDataset

batch = 60
training_data = TensorDataset(torch.from_numpy(dataset), torch.from_numpy(y).long())
train_loader = DataLoader(training_data, batch_size=batch, shuffle=True)

val_data = TensorDataset(torch.from_numpy(dataset2), torch.from_numpy(y2).long())
val_loader = DataLoader(val_data, batch_size=vlShape, shuffle=False)

test_data = TensorDataset(torch.from_numpy(dataset3), torch.from_numpy(y3).long())
test_loader = DataLoader(test_data, batch_size=testShape, shuffle=False)

In [5]:
# Evaluation function to calculate accuracy
def evaluate_accuracy(model, data_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # No gradient calculation during evaluation
        for inputs, labels in data_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)  # Get the predicted class
            correct += (predicted == labels).sum().item()  # Count correct predictions
            total += labels.size(0)  # Total samples
    
    accuracy = correct / total * 100  # Calculate accuracy as a percentage
    return accuracy

In [29]:
from lib.model import GPES

# # data params
# dataset = 'MBvsFMB.npy'
# train_X, train_y, test_X, test_y = np.load(os.path.join('.', 'data', dataset), allow_pickle=True)
# train_samples, L1 = train_X.shape

# model params
L2 = L1 - 1
alpha = L2
level = 5
number_classes = 2
C = 8e2
a = 1
K = 30

# fit params
batch_size = 60 # int(0.25 * train_samples)
lr = 8e-2
epochs = 1500

configs = {
    'data': {
        'dataset': dataset,
        'L1': L1,
    },
    'model': {
        'L2': L2,
        'alpha': alpha, 
        'level': level, 
        'C': C, 
        'a': a, 
        'K': K
    }, 
    'fit': {
        'batch_size': batch_size,
        'lr': lr,
        'epochs': epochs
    }
}

training_dir = os.path.join('checkpoints', 'run_004')
os.makedirs(training_dir, exist_ok=True)

# Initialize model, loss function, and optimizer
model = GPES(
    L1=L1, 
    L2=L2, 
    dim=dim, 
    order=order, 
    extended_order=extended_order, 
    alpha=alpha, 
    level=level, 
    number_classes=number_classes, 
    C=C, 
    a=a, 
    K=K
)

In [9]:
for inputs, labels in train_loader:
    if not np.isfinite(model(inputs).detach().numpy()).all():
        break

In [ ]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import os

training_dir = os.path.join('checkpoints', 'run_001')
os.makedirs(training_dir, exist_ok=True)

# Initialize loss function, and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

# Training parameters
num_epochs = 300
patience = 10

# Define the path to save the checkpoint
checkpoint_path = os.path.join(training_dir, 'checkpoint.pth')

# Initialize best_loss and early_stopping_counter
best_loss = float('inf')
early_stopping_counter = 0

# Load from checkpoint if it exists
if os.path.isfile(checkpoint_path):
    print("Loading checkpoint...")
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    best_loss = checkpoint['best_loss']
    early_stopping_counter = checkpoint['early_stopping_counter']
else:
    start_epoch = 0  # Start from the beginning if no checkpoint exists

exit = False

# Training loop
for epoch in range(start_epoch, num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    
    for inputs, labels in tqdm(train_loader):
        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(inputs)  # Forward pass
        if not np.isfinite(model(inputs).detach().numpy()).all():
            exit = True
            break
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize weights
        
        running_loss += loss.item()  # Accumulate loss

    if exit:
        break
    
    # Average loss for the training epoch
    train_loss = running_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}')

    # Validation step
    model.eval()  # Set the model to evaluation mode
    val_running_loss = 0.0
    with torch.no_grad():  # No gradient calculation for validation
        for inputs, labels in val_loader:
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute validation loss
            val_running_loss += loss.item()  # Accumulate validation loss

    # Average loss for the validation epoch
    val_loss = val_running_loss / len(val_loader)
    print(f'Validation Loss: {val_loss:.4f}')

    # Calculate and print accuracy on validation set
    accuracy = evaluate_accuracy(model, val_loader)
    print(f'Validation Accuracy: {accuracy:.2f}%')
    
    # Early stopping
    if val_loss < best_loss:
        best_loss = val_loss
        early_stopping_counter = 0  # Reset counter if loss improves
        print("Improved! Saving model...")
        torch.save(model.state_dict(), os.path.join(training_dir, 'best_model.pth'))  # Save the best model
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= patience:
            print("Early stopping triggered.")
            break  # Stop training if patience is exceeded

    # Save checkpoint after each epoch
    torch.save({
        'epoch': epoch + 1,  # Save next epoch
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_loss': best_loss,
        'early_stopping_counter': early_stopping_counter,
    }, checkpoint_path)

print("Training completed.")

In [23]:
training_dir = os.path.join('checkpoints', 'run_001')

# Load the best model weights
model.load_state_dict(torch.load(os.path.join(training_dir, 'best_model.pth')))

# Calculate and print accuracy on validation set
accuracy = evaluate_accuracy(model, train_loader)
print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 91.40%
